# データのロード

In [1]:
import numpy as np
import pandas as pd

In [2]:
from Dataset import Dataset
dataset = Dataset("Data/yelp")

# """
#         self.trainMatrix = self.load_training_file_as_matrix(path + ".train.rating")
#         self.trainList = self.load_training_file_as_list(path + ".train.rating")
#         self.testRatings = self.load_rating_file_as_list(path + ".test.rating") #
#         self.testNegatives = self.load_negative_file(path + ".test.negative")
        
#         trainMatrix：クラスDataの疎行列としての負荷評価レコード
#         trianList：評価レコードをリストとしてロードして、ユーザーの機能検索を高速化します。
#         testRatings：クラスEvaluateのリーブアウトワンアウト評価テスト
#         testNegatives：ユーザーによって評価されていない項目をサンプリングします
# """

already load the trainMatrix...
already load the trainList...


In [3]:
def sampling(dataset):
    _user_input, _item_input_pos = [], []
    for (u, i) in dataset.trainMatrix.keys():
        # positive instance
        _user_input.append(u)
        _item_input_pos.append(i)
    return _user_input, _item_input_pos

In [4]:
samples = sampling(dataset)

In [5]:
from multiprocessing import Pool
from multiprocessing import cpu_count

def shuffle(samples, batch_size, dataset):
    global _user_input
    global _item_input_pos
    global _batch_size
    global _index
    
    global _dataset
    _user_input, _item_input_pos = samples
    _batch_size = batch_size
    _dataset = dataset
    _index = range(len(_user_input))
    np.random.shuffle(_index)
    num_batch = len(_user_input) // _batch_size
    
    pool = Pool(cpu_count())
    res = pool.map(_get_train_batch, range(num_batch))
    pool.close()
    pool.join()
    
    user_list = [r[0] for r in res]
    item_pos_list = [r[1] for r in res]
    user_dns_list = [r[2] for r in res]
    item_dns_list = [r[3] for r in res]
    
    return user_list, item_pos_list, user_dns_list, item_dns_list

In [6]:
def _get_train_batch(i):
    user_batch, item_batch = [], []
    user_neg_batch, item_neg_batch = [], []
    
    begin = i * _batch_size
    
    for idx in range(begin, begin + _batch_size):
        
        user_batch.append(_user_input[_index[idx]])
        item_batch.append(_item_input_pos[_index[idx]])
        
        for dns in range(1):
            
            user = _user_input[_index[idx]]
            user_neg_batch.append(user)
            
            # negtive k
            j = np.random.randint(_dataset.num_items)
            
            while j in _dataset.trainList[_user_input[_index[idx]]]:
                j = np.random.randint(_dataset.num_items)
            item_neg_batch.append(j)
            
    return np.array(user_batch)[:, None], np.array(item_batch)[:, None], \
           np.array(user_neg_batch)[:, None], np.array(item_neg_batch)[:, None]

In [7]:
len(samples)

2

In [8]:
batches = shuffle(samples, 512, dataset)

In [9]:
train = batches

In [10]:
import torch
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
train_dataset = TensorDataset(torch.tensor(train, dtype=torch.int64))

In [10]:
# data = []
# for i in range(len(train_dataset[0][0])):
#     tmp = []
#     for j in range(4):
#         tmp.append(train_dataset[j][0][i])
#     data.append(tmp)

# train_dataset = data

In [ ]:
# train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)

#### train_dataset
* shape(4, samples)
  * 0: user_id
  * 1: item_pos
  * 2: user_id
  * 3: item_neg

# BPRのpytorch実装

#### ライブラリのインポート

In [12]:
import numpy as np
import pandas as pd
from torch import nn

In [13]:
def init_eval_model():
    pool = Pool(cpu_count())
    feed_dicts = pool.map(evaluate_input, range(dataset.num_users))
    pool.close()
    pool.join()
    return feed_dicts

def evaluate_input(user):
    # generate items_list
    test_item = dataset.testRatings[user][1]
    item_input = set(range(dataset.num_items)) - set(dataset.trainList[user])
    if test_item in item_input:
        item_input.remove(test_item)
    item_input = list(item_input)
    item_input.append(test_item)
    user_input = np.full(len(item_input), user, dtype='int32')[:, None]
    item_input = np.array(item_input)[:, None]
    return user_input, item_input

In [14]:
eval_d = init_eval_model()

#### モデルのクラスを定義

In [170]:
from __future__ import division
from multiprocessing import Pool
from multiprocessing import cpu_count
from torch import nn
import math
import torch.nn.functional as f

class APR(nn.Module):

    def __init__(self, n_user, n_item,  dataset, adv=1, feed_dicts=None, k=8):        
        super(APR, self).__init__()
        self.reg = 1 
        self.adv = adv
        self.reg_adv = 1
        self.eps = 0.5
        
        self.embedding_P = nn.Embedding(n_user, k)#.from_pretrained(emb_P)
        self.embedding_Q = nn.Embedding(n_item, k)#.from_pretrained(emb_Q)
        nn.init.normal_(self.embedding_P.weight, 0.0, 0.01)
        nn.init.normal_(self.embedding_Q.weight, 0.0, 0.01)
        
        # adversarial
        self.delta_P = nn.Embedding(n_user, k)
        self.delta_Q = nn.Embedding(n_item, k)
        
        self.delta_P .weight.requires_grad = False
        self.delta_Q .weight.requires_grad = False
        
        self.delta_P.weight = nn.Parameter(torch.zeros([n_user, k], dtype=torch.float32))
        self.delta_Q.weight = nn.Parameter(torch.zeros([n_user, k], dtype=torch.float32))
        
        self.h = torch.ones(k, 1)
        self.dataset = dataset
        self.eval_data =  feed_dicts
    
    def forward(self, user, item):        
        if self.adv == 0:
            return torch.matmul(self.embedding_P(user.flatten()) * self.embedding_Q(item.flatten()), self.h).sum(1)
        else:
#             print("aaa")
            ##adversarialな計算
            P_plus_delta = torch.add(self.embedding_P(user.flatten()), self.delta_P(user.flatten()))
            Q_plus_delta = self.embedding_Q(user.flatten()) + self.delta_Q(user.flatten())
            adv_term = torch.matmul(P_plus_delta * Q_plus_delta, self.h).sum(1)            
            return torch.matmul(self.embedding_P(user.flatten()) * self.embedding_Q(item.flatten()), self.h).sum(1), adv_term 
                
    def bpr_loss(self, user, items):
        pos_preds = self.predict(user, items[0])
        neg_preds = self.predict(user, items[1])
        preds = pos_preds - neg_preds
        loss = torch.log(1 + torch.exp(-preds)).sum()
        return loss
    
    def apr_loss(self, user, items):
        pos_preds_adv = self.predict_adv(user, items[0])
        neg_preds_adv = self.predict_adv(user, items[1])
        preds_adv = pos_preds_adv - neg_preds_adv 
        loss = self.reg_adv * torch.log(1 + torch.exp(-preds_adv)).sum()  
        return loss
    
    def predict(self, user, item):        
        return torch.matmul(self.embedding_P(user.flatten()) * self.embedding_Q(item.flatten()), self.h).sum(1)
    
    def predict_adv(self, user, item):  
        P_plus_delta = torch.add(self.embedding_P(user.flatten()), self.delta_P(user.flatten()))
        Q_plus_delta = torch.add(self.embedding_Q(user.flatten()), self.delta_Q(user.flatten()))
        return torch.matmul(P_plus_delta * Q_plus_delta, self.h).sum(1)
                
    def evaluate(self):
        res = []
        for user in range(self.dataset.num_users):
            res.append(self.eval_by_user(user))
        res = np.array(res)
        hr, ndcg, auc = (res.mean(axis=0)).tolist()
        return hr, ndcg, auc

    def eval_by_user(self, user):

        user_input, item_input = self.eval_data[user]

        predictions = self.predict(torch.tensor(user_input, dtype=torch.long), torch.tensor(item_input, dtype=torch.long))
        neg_predict, pos_predict = predictions[:-1], predictions[-1]
        position = (neg_predict >= pos_predict).sum()
        hr, ndcg, auc = [], [], []
        for k in range(1,100 + 1):
            hr.append(position < k)
            ndcg.append(math.log(2) / math.log(position + 2) if position < k else 0)
            auc.append(1 - (position / len(neg_predict)))

        return hr, ndcg, auc

In [171]:
from torch import optim
adv = 1
n_user = dataset.num_users
n_item = dataset.num_items
model = APR(n_user, n_item, dataset, adv, feed_dicts=eval_d)
optimizer = optim.SGD(model.parameters(), lr=5e-2)

In [172]:
batch_size = 512
batch_num = len(train_dataset[0][0]) // 512 + 1

In [176]:
from torch import autograd
model.train()
for epoch in range(3):
    print("epoch", epoch)
    for i in range(1312):
        
        model.zero_grad()

        user = autograd.Variable(train_dataset[0][0][i])
        item_pos = autograd.Variable(train_dataset[1][0][i])
        item_neg = autograd.Variable(train_dataset[3][0][i])
        
        user = user.view(-1,)
        item_pos = item_pos.view(-1,)
        item_neg = item_neg.view(-1,)
        

        loss_bpr = model.bpr_loss(user, (item_pos, item_neg))
        loss_bpr.backward()
        
        grad_P = model.embedding_P.weight.grad
        grad_Q = model.embedding_Q.weight.grad
        model.delta_P.weight =  nn.Parameter(model.eps * f.normalize(model.embedding_P.weight.grad, p=2, dim=1))
        model.delta_Q.weight =  nn.Parameter(model.eps * f.normalize(model.embedding_Q.weight.grad, p=2, dim=1))
        
        loss_apr = model.apr_loss(user, (item_pos, item_neg))
#         model.zero_grad()
        loss_apr.backward()
        # Update the parameters
        optimizer.step()
        
    print(model.state_dict())
    print(loss)

print("eval")
result = model.evaluate()
hr, ndcg, auc = np.swapaxes(result, 0, 1)[-1]
print(hr, ndcg, auc)

('epoch', 0)
OrderedDict([('embedding_P.weight', tensor([[ 0.0136, -0.0127, -0.0041,  ..., -0.0166,  0.0077, -0.0093],
        [ 0.0079,  0.0028, -0.0204,  ..., -0.0264,  0.0182, -0.0037],
        [ 0.0162, -0.0018,  0.0103,  ..., -0.0082, -0.0049,  0.0065],
        ...,
        [ 0.0045,  0.0076, -0.0032,  ..., -0.0380,  0.0009, -0.0106],
        [ 0.0089, -0.0063,  0.0002,  ...,  0.0042,  0.0111,  0.0008],
        [-0.0034,  0.0050, -0.0103,  ...,  0.0071, -0.0031, -0.0184]])), ('embedding_Q.weight', tensor([[-0.0046,  0.0013,  0.0074,  ..., -0.0097,  0.0099,  0.0213],
        [ 0.0005, -0.0055, -0.0036,  ..., -0.0084,  0.0148,  0.0167],
        [-0.0006,  0.0021, -0.0013,  ..., -0.0186,  0.0252,  0.0108],
        ...,
        [-0.0056, -0.0200,  0.0218,  ..., -0.0055,  0.0076,  0.0064],
        [-0.0014, -0.0123,  0.0103,  ..., -0.0041, -0.0014,  0.0177],
        [-0.0032, -0.0024, -0.0064,  ...,  0.0003, -0.0017, -0.0046]])), ('delta_P.weight', tensor([[0., 0., 0.,  ..., 0., 0., 0.

# 評価関数定義